In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_pickle('M1_HISTORICAL_DATA.pkl')

In [4]:
data.drop('real_volume', axis=1, inplace=True)

In [14]:
data.head()

,time,open,high,low,close,tick_volume,spread,MA5,MA8,MA13,MA20,RSI-4,RSI-8
19,2023-01-02 09:19:00,1.06801,1.06802,1.06799,1.06799,4,77,1.068018,1.068031,1.068165,1.068301,22.222222,41.463415
20,2023-01-02 09:20:00,1.06800,1.06807,1.06800,1.06804,11,72,1.067998,1.068027,1.068135,1.068273,78.571429,46.666667
21,2023-01-02 09:21:00,1.06807,1.06808,1.06804,1.06807,10,68,1.068020,1.068030,1.068104,1.068248,80.000000,52.173913
22,2023-01-02 09:22:00,1.06808,1.06819,1.06807,1.06819,17,65,1.068062,1.068049,1.068078,1.068229,86.956522,63.157895
23,2023-01-02 09:23:00,1.06818,1.06821,1.06816,1.06821,5,67,1.068100,1.068058,1.068063,1.068210,100.000000,57.142857


In [6]:
# Calculate Moving Averages (20 and 50 periods)
data['MA5'] = data['close'].rolling(window=5).mean()
data['MA8'] = data['close'].rolling(window=8).mean()
data['MA13'] = data['close'].rolling(window=13).mean()
data['MA20'] = data['close'].rolling(window=20).mean()
# Calculate RSI
def calculate_rsi(data, periods=4):
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

data['RSI-4'] = calculate_rsi(data)
data['RSI-8'] = calculate_rsi(data, 8)

In [11]:
data.dropna(inplace=True)

In [13]:
data.count()

time           341271
open           341271
high           341271
low            341271
close          341271
tick_volume    341271
spread         341271
MA5            341271
MA8            341271
MA13           341271
MA20           341271
RSI-4          341271
RSI-8          341271
dtype: int64